<a href="https://colab.research.google.com/github/ancestor9/2025_Spring_Capstone-Design/blob/main/Week06/MCP_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Step 1: 필요한 라이브러리 설치
!pip install streamlit pyngrok praisonaiagents --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.9 MB/s eta 0:00:00


In [6]:
# Step 2: streamlit_app.py 파일 작성
code = '''
import streamlit as st
from praisonaiagents import Agent, MCP

st.title("🏠 Airbnb Booking Assistant")

@st.cache_resource
def get_agent():
    return Agent(
        instructions="You help book apartments on Airbnb.",
        llm="gpt-4o-mini",
        tools=MCP("npx -y @openbnb/mcp-server-airbnb --ignore-robots-txt")
    )

if "messages" not in st.session_state:
    st.session_state.messages = []

for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

with st.form("booking_form"):
    st.subheader("Enter your booking details")

    destination = st.text_input("Destination:", "Paris")
    col1, col2 = st.columns(2)
    with col1:
        check_in = st.date_input("Check-in date")
    with col2:
        check_out = st.date_input("Check-out date")
    adults = st.number_input("Number of adults:", min_value=1, max_value=10, value=2)

    submitted = st.form_submit_button("Search for accommodations")

    if submitted:
        search_agent = get_agent()
        query = f"I want to book an apartment in {destination} from {check_in.strftime('%m/%d/%Y')} to {check_out.strftime('%m/%d/%Y')} for {adults} adults"
        st.session_state.messages.append({"role": "user", "content": query})
        with st.chat_message("user"):
            st.markdown(query)
        with st.chat_message("assistant"):
            with st.spinner("Searching for accommodations..."):
                response = search_agent.start(query)
                st.markdown(response)
        st.session_state.messages.append({"role": "assistant", "content": response})

if st.session_state.messages:
    prompt = st.chat_input("Ask a follow-up question about the accommodations")
    if prompt:
        search_agent = get_agent()
        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)
        with st.chat_message("assistant"):
            with st.spinner("Thinking..."):
                response = search_agent.start(prompt)
                st.markdown(response)
        st.session_state.messages.append({"role": "assistant", "content": response})
'''

with open("streamlit_app.py", "w") as f:
    f.write(code)


In [7]:
from google.colab import userdata

# ngrok 인증 토큰 불러오기
ngrok_token = userdata.get('ngrok')

# ngrok에 인증 토큰 등록
!ngrok config add-authtoken {ngrok_token}


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [8]:
from pyngrok import ngrok

ngrok.kill()  # 혹시 모를 세션 종료

# 정확한 포맷: addr 파라미터 사용
public_url = ngrok.connect(addr="8501", proto="http")
print(f"✅ Streamlit App is running: {public_url}")

# 스트림릿 앱 실행
!streamlit run streamlit_app.py &>/dev/null &


✅ Streamlit App is running: NgrokTunnel: "https://f5e8-34-16-95-210.ngrok-free.app" -> "http://localhost:8501"
